# Software-Engineering
# Machine Learning Model

## Purpose
The purpose of the machine learning model is to predict the features according to the training sets that we get from the dublin bike api and google weather api.

 - to get the number of available bike of one specific station
 - to gat the number of available bike stands of one specific station

## Import the packages

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns


from sqlalchemy import create_engine
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics
pd.options.mode.chained_assignment=None
%matplotlib inline
import os
import pymysql
pymysql.install_as_MySQLdb()
import pandas as pd
import sys

## Connect to the RDS database

In [2]:
def connect_to_database():
                                                                       
    conn = pymysql.connect(
        host="group20db2.cuvpbui26dwd.eu-west-1.rds.amazonaws.com",  # mysql服务器地址
        port=3306,  # 端口号
        user="group20",  # 用户名
        passwd="comp30830",  # 密码
        db="segroupproject",  # 数据库名称
        charset='utf8',  # 连接编码，根据需要填写
    )
    return conn

try:
    conn = connect_to_database()
    cur = conn.cursor()
    print('--- CONNECTED TO DATABASE SUCCESSFULLY ---')
    
    
# Exit if connection not working   
except Exception as e:
        sys.exit(e)

    

--- CONNECTED TO DATABASE SUCCESSFULLY ---


## Get data from the RDS database
### Get the bike information from the bike table

In [4]:
All_info = pd.read_sql_query('SELECT * FROM segroupproject.dynamic_bikeData, segroupproject.weather WHERE weather.current - dynamic_bikeData.now_time <150 AND weather.current - dynamic_bikeData.now_time>0 AND dynamic_bikeData.now_time > "2020-03-24 18:00:27" AND weather.current > "2020-03-24 18:01";', conn)
All_info.head(20)



# AllStations_AllWeather = 'SELECT * FROM segroupproject.dynamic_bikeData, segroupproject.weather WHERE weather.dateTime - dynamic_bikeData.dateTime < 5 AND dynamic_bikeData.dateTime > "2020-03-24 18:00:27" AND weather.dateTime > "2020-03-24 18:01" ;'








,number,status,available_bike_stands,available_bikes,last_update,now_time,current,lon,lat,weather_id,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod


In [3]:
df = pd.read_sql_query("SELECT * FROM segroupproject.dynamic_bikeData", conn)

# Examine dataframe object, show first 10 rows
df.head(10)

,number,status,available_bike_stands,available_bikes,last_update,now_time
0,42,OPEN,27,3,2020-03-05 10:25:33,2020-03-05 10:28:01
1,30,OPEN,17,3,2020-03-05 10:20:14,2020-03-05 10:28:01
2,54,OPEN,2,31,2020-03-05 10:22:53,2020-03-05 10:28:01
3,108,OPEN,40,0,2020-03-05 10:23:32,2020-03-05 10:28:01
4,56,OPEN,0,40,2020-03-05 10:21:31,2020-03-05 10:28:01
5,6,OPEN,17,3,2020-03-05 10:18:19,2020-03-05 10:28:01
6,18,OPEN,26,4,2020-03-05 10:24:59,2020-03-05 10:28:01
7,32,OPEN,3,27,2020-03-05 10:24:38,2020-03-05 10:28:01
8,52,OPEN,27,5,2020-03-05 10:21:12,2020-03-05 10:28:01
9,48,OPEN,18,22,2020-03-05 10:25:12,2020-03-05 10:28:01


In [4]:
#show the dataframe has how many rows and columns
print("The dataset has %s rows and %s columns." % df.shape)

The dataset has 628381 rows and 6 columns.


### Get the weather information from the weather table

In [7]:
# Create dataframe and store data running SQL query
df_2 = pd.read_sql_query("SELECT * FROM segroupproject.weather", conn)

# Examine dataframe object, show first 50 rows
df_2.head(10)

,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod
0,2020-03-05 10:52,-6.27,53.34,802,Clouds,scattered clouds,03d,stations,3.86,0.51,...,1583404828,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
1,2020-03-05 11:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,4.29,-0.39,...,1583407215,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
2,2020-03-05 11:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,4.47,-0.17,...,1583408505,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
3,2020-03-05 12:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,5.31,0.82,...,1583410536,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
4,2020-03-05 12:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,5.48,1.03,...,1583412184,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
5,2020-03-05 13:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,6.32,1.81,...,1583414286,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
6,2020-03-05 13:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,6.34,1.68,...,1583416204,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
7,2020-03-05 14:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,6.79,2.21,...,1583417827,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
8,2020-03-05 14:52,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.15,3.32,...,1583419704,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200
9,2020-03-05 15:22,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.06,3.21,...,1583421692,1,1565,IE,1583391765,1583431833,0,2964574,Dublin,200


In [10]:
df = df[(pd.to_datetime(df['now_time'] ,format = '%Y-%m-%d %H:%M:%S')>= pd.to_datetime('2020-03-24 18:00:27',format = '%Y-%m-%d %H:%M:%S'))]
df = df.reset_index(drop=True)
df


,number,status,available_bike_stands,available_bikes,last_update,now_time
0,42,OPEN,18,11,2020-03-24 17:59:20,2020-03-24 18:00:27
1,30,OPEN,8,12,2020-03-24 17:52:44,2020-03-24 18:00:27
2,54,OPEN,16,17,2020-03-24 17:54:32,2020-03-24 18:00:27
3,108,OPEN,23,17,2020-03-24 17:53:16,2020-03-24 18:00:27
4,56,OPEN,30,10,2020-03-24 17:57:14,2020-03-24 18:00:27
...,...,...,...,...,...,...
224395,39,OPEN,8,12,2020-03-31 20:26:21,2020-03-31 20:32:16
224396,83,OPEN,23,17,2020-03-31 20:27:38,2020-03-31 20:32:16
224397,92,OPEN,28,12,2020-03-31 20:27:29,2020-03-31 20:32:16
224398,21,OPEN,19,11,2020-03-31 20:29:29,2020-03-31 20:32:16


In [ ]:
df_group = df.groupby(['now_time'])

for index1, data in df_group:
    print(index1,data)
    

In [11]:
df_2 = df_2[(pd.to_datetime(df_2['current'] ,format = '%Y-%m-%d %H:%M')>= pd.to_datetime('2020-03-24 18:01',format = '%Y-%m-%d %H:%M'))]
df_2 = df_2.reset_index(drop=True)
df_2


,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
1,2020-03-24 18:06,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
2,2020-03-24 18:11,-6.27,53.34,803,Clouds,broken clouds,04d,stations,10.84,4.1,...,1585073453,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
3,2020-03-24 18:16,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585073019,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
4,2020-03-24 18:21,-6.27,53.34,803,Clouds,broken clouds,04d,stations,10.84,4.1,...,1585073453,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2043,2020-03-31 20:15,-6.27,53.34,803,Clouds,broken clouds,04n,stations,7.18,2.65,...,1585685692,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200
2044,2020-03-31 20:20,-6.27,53.34,803,Clouds,broken clouds,04n,stations,7.18,2.65,...,1585685756,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200
2045,2020-03-31 20:25,-6.27,53.34,803,Clouds,broken clouds,04n,stations,7.18,2.65,...,1585686136,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200
2046,2020-03-31 20:30,-6.27,53.34,803,Clouds,broken clouds,04n,stations,7.18,2.65,...,1585686312,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200


In [31]:
df_2['current'].value_counts()

2020-03-31 05:44    1
2020-03-30 20:34    1
2020-03-26 15:32    1
2020-03-27 05:07    1
2020-03-30 05:49    1
                   ..
2020-03-27 23:28    1
2020-03-29 03:43    1
2020-03-31 03:54    1
2020-03-26 16:27    1
2020-03-30 01:29    1
Name: current, Length: 2024, dtype: int64

In [24]:
df_3 = pd.DataFrame()
for i in range(len(df_2)):
    a=df_2.loc[i]
    d=pd.DataFrame(a).T
    df_3=df_3.append([d]*110)
df_3

,current,lon,lat,weather_id,main,description,icon,base,temp,feels_like,...,dt,sys_type,sys_id,sys_country,sunrise,sunset,timezone,id,name,cod
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
0,2020-03-24 18:01,-6.27,53.34,803,Clouds,broken clouds,04d,stations,11.22,3.85,...,1585072763,1,1565,IE,1585030617,1585075523,0,2964574,Dublin,200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023,2020-03-31 18:35,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.85,4.3,...,1585679429,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200
2023,2020-03-31 18:35,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.85,4.3,...,1585679429,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200
2023,2020-03-31 18:35,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.85,4.3,...,1585679429,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200
2023,2020-03-31 18:35,-6.27,53.34,803,Clouds,broken clouds,04d,stations,7.85,4.3,...,1585679429,1,1565,IE,1585634397,1585681080,3600,2964574,Dublin,200


In [8]:
df = df['now_time']
df

0         2020-03-24 18:00:27
1         2020-03-24 18:00:27
2         2020-03-24 18:00:27
3         2020-03-24 18:00:27
4         2020-03-24 18:00:27
                 ...         
216695    2020-03-31 14:41:19
216696    2020-03-31 14:41:19
216697    2020-03-31 14:41:19
216698    2020-03-31 14:41:19
216699    2020-03-31 14:41:19
Name: now_time, Length: 216700, dtype: object

In [13]:
mergedStuff = pd.merge(df, df_2, left_on = 'current', how = 'outer')
mergedStuff

TypeError: object of type 'NoneType' has no len()

In [ ]:
d = pd.DataFrame({'A':{'1':'01','2':'02','3':'03','4':'01','5':'02','6':'03','7':'01','8':'02','9':'03'},'ID':{'1':'10','2':'10','3':'10','4':'15','5':'15','6':'15','7':'20','8':'20','9':'20'},'C':{'1':'10A','2':'10A','3':'10A','4':'15A','5':'15A','6':'15A','7':'20A','8':'20A','9':'20A'}, })
d['D'] = d['ID']
d





In [23]:
f = pd.DataFrame({'ID':{'1':'10A','2':'15A','3':'20A'}})
o = pd.DataFrame()
for i in range(len(f)):
    a=f.iloc[i]
    k=pd.DataFrame(a).T
    o=o.append([k]*5)
o 
   

,ID
1,10A
1,10A
1,10A
1,10A
1,10A
2,15A
2,15A
2,15A
2,15A
2,15A


In [24]:
f.iloc[0]['ID']

'10A'

In [6]:
d.merge(f, how='left', on="ID")

,A,ID,D
0,01,10,10
1,02,10,10
2,03,10,10
3,01,15,15
4,02,15,15
5,03,15,15
6,01,20,20
7,02,20,20
8,03,20,20


In [4]:
def join(d1,d2):
    count =0
    for i in range(0, len(d1)-1):
        if d1.iloc[i]['D'] == d1.iloc[i+1]['D']:
            d1.loc[i,'D'] = d2.iloc[count]['ID']
            print(d1.iloc[i]['D'],d2.iloc[count]['ID'])
        else:
            d1.loc[i,'D'] = d2.iloc[count]['ID']
            count += 1
#         if d1.iloc[i]['ID'] == d1.iloc[i+1]['ID']:
#             d1.iloc[i]['D'] = d2.iloc[count]['ID']
#             print(d1.loc[i]['D'])
#         else:
#             count+=1
            
join(d,f)


10 10A
10 10A
15 15A
15 15A
20 20A
20 20A


In [5]:
d


,A,ID,D
1,01,10,10
2,02,10,10
3,03,10,10
4,01,15,15
5,02,15,15
6,03,15,15
7,01,20,20
8,02,20,20
9,03,20,20
0,NaN,NaN,10A


In [68]:
a = pd.merge(d, f,left_index=True, right_index=True, how='inner')
a

,A,B_x,B_y
1,01,10,10A
2,02,10,15A
3,03,10,20A


In [66]:
a = [(d, f) for k, l in zip(ntest, ltest) for v in l]

NameError: name 'ntest' is not defined